## basic analysis of mag-comp bauer model

* ....
* model comparison

In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import os
import arviz as az

bids_folder = '/Users/mrenke/data/ds-dnumrisk'
target_folder = op.join(bids_folder, f'derivatives/cogmodels_magjudge')

sns.set_context('talk')


In [14]:
from utils import get_data

df = get_data(bids_folder)

number of subjects found: 66
number of subjects in dataframe: 66
Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66],
           dtype='int64', name='subject')


## Model Comparison

In [3]:
target_folder = op.join(bids_folder, f'derivatives/cogmodels_magjudge')

model_1 =  az.from_netcdf(op.join(target_folder, f'model-1_trace.netcdf'))
model_2 =  az.from_netcdf(op.join(target_folder, f'model-2_trace.netcdf'))
model_3 =  az.from_netcdf(op.join(target_folder, f'model-3_trace.netcdf'))
model_4 =  az.from_netcdf(op.join(target_folder, f'model-4_trace.netcdf'))
model_5 =  az.from_netcdf(op.join(target_folder, f'model-5_trace.netcdf'))


In [4]:
compare_dict = {"model 1": model_1, "model 2": model_2,"model 3": model_3, "model 4": model_4}
compare_dict = {"n1/n2, no fit prior": model_1, "n1/n2, fit prior": model_2,
                "per/mem noise, no fit prior": model_3, "per/mem noise, fit prior": model_4, 
                 "per/mem noise, fit prior, only prior_sd group reg": model_5}

az.compare(compare_dict)

/Users/mrenke/mambaforge/envs/behav_fit/lib/python3.10/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/Users/mrenke/mambaforge/envs/behav_fit/lib/python3.10/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/Users/mrenke/mambaforge/envs/

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
"per/mem noise, fit prior, only prior_sd group reg",0,-3729.604808,131.117813,0.000000,7.319256e-01,63.769682,0.000000,True,log
"per/mem noise, fit prior",1,-3730.087301,131.803883,0.482493,0.000000e+00,63.797268,0.813595,True,log
"n1/n2, fit prior",2,-3733.050611,134.136775,3.445803,2.244487e-01,63.652330,3.938976,True,log
"n1/n2, no fit prior",3,-3797.628828,89.875935,68.024020,4.362577e-02,63.361328,12.139295,True,log
"per/mem noise, no fit prior",4,-3804.242526,75.191898,74.637718,7.793766e-14,63.557526,12.000286,True,log


### 

### Get subwise estimates



In [4]:
from utils_02 import get_subwise_params

idata =  az.from_netcdf(op.join(target_folder, f'model-5_trace.netcdf'))

sd_mem = get_subwise_params(idata, 'memory_noise_sd')
sd_per = get_subwise_params(idata, 'perceptual_noise_sd')


In [9]:
from utils_02 import build_model
model = build_model('5',df=df)
model.build_estimation_model()

model_5_idata =  az.from_netcdf(op.join(target_folder, f'model-5_trace.netcdf'))

sd_per = model.get_subjectwise_parameter_estimates(idata=model_5_idata, parameters='memory_noise_sd')

: 

In [4]:
from bauer.utils.bayes import softplus

df_params = sd_mem.join(sd_per, on='subject')

for col in df_params.columns:
    df_params[col] = softplus(df_params[col])
df_params.head()

NameError: name 'sd_mem' is not defined

In [8]:
((np.log(df.n1).mean()+np.log(df.n2).mean()) / 2)



2.2629271978496392

In [12]:
np.log(df.n2).mean()

2.2262752634515124

In [7]:
np.log((df.n1.mean()+df.n2.mean()) / 2)


2.4332434044372904

### ppcs

In [32]:
model.free_parameters

{'n1_evidence_sd': {'mu_intercept': -1.0, 'transform': 'softplus'},
 'n2_evidence_sd': {'mu_intercept': -1.0, 'transform': 'softplus'}}

In [23]:
variable_names = list(idata.posterior.data_vars.keys())
variable_names

['n1_evidence_sd_mu',
 'n1_evidence_sd_offset',
 'n2_evidence_sd_mu',
 'n2_evidence_sd_offset',
 'n1_evidence_sd_sd',
 'n2_evidence_sd_sd',
 'n1_evidence_sd',
 'n2_evidence_sd']

In [31]:
n1 = softplus(idata['posterior']['n1_evidence_sd_mu']).to_dataframe().reset_index('n1_evidence_sd_regressors').rename(mapper={'n1_evidence_sd_regressors':'regressor'}, axis=1)
n1.head()

regressor  n1_evidence_sd_mu
chain draw                              
0     0     Intercept           0.227168
      0         group           0.864315
      1     Intercept           0.223017
      1         group           0.746213
      2     Intercept           0.236774